In [3]:
import tensorflow as tf
import numpy as np
#trackback error 생성시 넣어주는 함수 그래프를 리셋시켜줌.
tf.reset_default_graph()

# one-hot encoding 으로 표현한 알파벳 값
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

# 텐서 char_arr의 데이터를 배열로 저장
# 해당 글자의 인덱스를 구할 수 있는 연관배열(딕셔너리)생성
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 학습에 사용될 배열 저장
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

# 학습에 사용될 단어들을 사용될 수 있도록 변환 해 주는 함수.
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        # 입력값용 배열생성 : seq_data에 있는 단어들의 앞 세 글자 알파벳 인덱스를 구한 배열을 만든다.
        input = [num_dic[n] for n in seq[:-1]]
        
        # 출력값 : 단어들의 마지막 알파벳 인덱스를 구함.
        target = num_dic[seq[-1]]
        
        # 입력값을 원-핫 인코딩으로 변환시킵니다.
        # 단어가 "deep"이라면, 입력으로 d,e,e를 취하고 배열은 [3,4,4]가 나온다.
        # 이를 원-핫 인코딩하면 [[0. 0. 0. 1. 0. 0. 0. ... 0.]
        #                        [0. 0. 0. 0. 1. 0. 0. ... 0.]
        #                        [0. 0. 0. 0. 1. 0. 0. ... 0.]]
        # 출력값(실측값)은 "p"가 되는데, target에 들어있는 값은 15가 될 것입니다.
        input_batch.append(np.eye(dic_len)[input])
        # 지금까지 손실함수로 사용하던 softmax_cross_entropy_with_logits 함수는
        # label 값을 one-hot 인코딩으로 넘겨줘야 하지만,
        # 이 예제에서 사용할 손실 함수인 sparse_softmax_cross_entropy_with_logits 는
        # one-hot 인코딩을 사용하지 않으므로 index 를 그냥 넘겨주면 됩니다.
        target_batch.append(target)
        
    return input_batch, target_batch


##########
#옵션 설정
##########
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
# 타입 스텝: [1 2 3] => 3
# RNN 을 구성하는 시퀀스(단계)의 갯수입니다.
n_step = 3
# 입력값 크기. 알파벳에 대한 one-hot 인코딩이므로 26개가 됩니다.
# 예) c => [0 0 1 0 0 0 0 0 0 0 0 ... 0]
# 출력값도 입력값과 마찬가지로 26개의 알파벳으로 분류합니다.
n_input = n_class = dic_len

#################
#신경망 모델 구성
#################
X = tf.placeholder(tf.float32, [None, n_step, n_input])
# 비용함수에 sparse_softmax_cross_entropy_with_logits 을 사용하므로
# 출력값과의 계산을 위한 원본값의 형태는 one-hot vector가 아니라 인덱스 숫자를 그대로 사용하기 때문에
# 다음처럼 하나의 값만 있는 1차원 배열을 입력값으로 받습니다.
# [3] [3] [15] [4] ...
# 기존처럼 one-hot 인코딩을 사용한다면 입력값의 형태는 [None, n_class] 여야합니다.
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

# RNN 셀을 생성합니다.
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# 과적합 방지를 위한 Dropout 기법을 사용합니다.
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
# 여러개의 셀을 조합해서 사용하기 위해 셀을 추가로 생성합니다.
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# 앞에 만든 두 개의 셀을 조합시킴
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
# tf.nn.dynamic_rnn()함수로 심층 순환 신경망 Deep RNN을 만든다.
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

# 최종 출력층 생성
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

# 손실 함수, 최적화 함수 사용
cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

#########
#학습시작
#########
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# make_batch()함수를 이용해 seq_data에 저장한 단어들을 입력값과 실측값으로 분리한다.
input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       # 분리한 입력값과 실측값을 이용해 학습시킨다.
                       feed_dict={X: input_batch, Y: target_batch})
          
    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')
##############
#학습결과 확인
##############

# 결괏값으로 예측한 단어를 정확도와 함께 출력하도록 코딩
# 실측값(model) = Y는 정수 이기때문에 int32로 변환해준다
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

#학습에 사용한 단어를 넣고 예측모델 구동
input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

# 모델이 예측한 값을 가지고, 각각의 값에 해당하는 인덱스의 알파벳을 가져와서 예측한 단어를 출력한다.
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)

Epoch: 0001 cost = 3.894518
Epoch: 0002 cost = 2.863219
Epoch: 0003 cost = 1.416545
Epoch: 0004 cost = 1.904375
Epoch: 0005 cost = 1.021528
Epoch: 0006 cost = 0.877895
Epoch: 0007 cost = 0.849619
Epoch: 0008 cost = 0.867587
Epoch: 0009 cost = 0.548594
Epoch: 0010 cost = 0.489727
Epoch: 0011 cost = 0.360291
Epoch: 0012 cost = 0.301956
Epoch: 0013 cost = 0.203523
Epoch: 0014 cost = 0.198454
Epoch: 0015 cost = 0.192430
Epoch: 0016 cost = 0.220745
Epoch: 0017 cost = 0.068314
Epoch: 0018 cost = 0.113156
Epoch: 0019 cost = 0.226180
Epoch: 0020 cost = 0.189144
Epoch: 0021 cost = 0.076474
Epoch: 0022 cost = 0.212360
Epoch: 0023 cost = 0.120648
Epoch: 0024 cost = 0.074244
Epoch: 0025 cost = 0.017856
Epoch: 0026 cost = 0.048045
Epoch: 0027 cost = 0.017446
Epoch: 0028 cost = 0.009041
Epoch: 0029 cost = 0.015694
Epoch: 0030 cost = 0.025940
최적화 완료!

=== 예측 결과 ===
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool